In [24]:
import duckdb

# some DuckDB setup
con = duckdb.connect(database=':memory:')
# enable automatic query parallelization
con.execute("PRAGMA threads=2")
# enable caching of parquet metadata
con.execute("PRAGMA enable_object_cache")

In [25]:
import numpy as np

# Load all the data into the duck database
all_training_data = con.execute('''
    SELECT 
        tconst,
        
        -- Clean up the movie title text. Remove excess whitespace, convert to lowercase, convert non-ascii to ascii equivalent, 
        -- remove everything that is non-alpanumeric or a space.
        REGEXP_REPLACE(TRANSLATE(LOWER(TRIM(primaryTitle)), 'áàãäåæßçéèêíîïñòóôöøớúûüý','aaaaaabceeeiiinoooooouuuy'),'[^a-zA-Z0-9 ]','','g') AS pTitle,
        REGEXP_REPLACE(TRANSLATE(LOWER(TRIM(originalTitle)),'áàãäåæßçéèêíîïñòóôöøớúûüý','aaaaaabceeeiiinoooooouuuy'),'[^a-zA-Z0-9 ]','','g') AS oTitle,
        
        -- Flag for indicating foreign film
        CASE
           WHEN oTitle IS NULL THEN 0
           ELSE 1
        END AS ForeignFilm,
        
        -- Count number of words in title
        LENGTH(pTitle) - LENGTH(REPLACE(pTitle, ' ', '')) + 1 AS n_words,
        
        -- Merge start year and end year into single column
        CASE
            WHEN startYear LIKE '%N%' THEN endYear
            ELSE startYear
        END AS Year,
        runtimeMinutes,
        numVotes,
        
        -- Convert True/False to 1/0
        CAST(label AS INT) as label
    FROM 'big-data-course-2024-projects/imdb/train-[1-8].csv'
''').df()

# Replace empty (\N) with nans
all_training_data = all_training_data.replace('\\N', np.nan)

all_training_data

,tconst,pTitle,oTitle,ForeignFilm,n_words,Year,runtimeMinutes,numVotes,label
0,tt0010600,the doll,die puppe,1,2,1919,66,1898.0,1
1,tt0011841,way down east,way down east,1,3,1920,145,5376.0,1
2,tt0012494,destiny,der mude tod,1,1,1921,97,5842.0,1
3,tt0015163,the navigator,the navigator,1,2,1924,59,9652.0,1
4,tt0016220,the phantom of the opera,the phantom of the opera,1,5,1925,93,17887.0,1
...,...,...,...,...,...,...,...,...,...
7954,tt9625664,trauma center,None,0,2,2019,87,12951.0,0
7955,tt9741310,slaxx,slaxx,1,1,2020,77,2464.0,0
7956,tt9742392,kindred,kindred,1,1,2020,101,1719.0,0
7957,tt9850386,the bee gees how can you mend a broken heart,None,0,10,2020,111,4144.0,1


In [26]:
import pandas as pd

# Load json files and drop empty rows
directors = pd.read_json('big-data-course-2024-projects/imdb/directing.json').replace('\\N', np.nan).dropna()
writers = pd.read_json('big-data-course-2024-projects/imdb/writing.json').replace('\\N', np.nan).dropna()

# Combine all films listed in the writer/director files with the year of the film 
# (from training, validation and test data).
movie_year = con.execute('''
    SELECT tconst,
        CASE
            WHEN startYear LIKE '%N%' THEN endYear
            ELSE startYear
        END AS Year,
    FROM 'big-data-course-2024-projects/imdb/train-[1-8].csv'
    UNION
    SELECT tconst,
        CASE
            WHEN startYear LIKE '%N%' THEN endYear
            ELSE startYear
        END AS Year,
    FROM 'big-data-course-2024-projects/imdb/test_hidden.csv'
    UNION
    SELECT tconst,
        CASE
            WHEN startYear LIKE '%N%' THEN endYear
            ELSE startYear
        END AS Year,
    FROM 'big-data-course-2024-projects/imdb/validation_hidden.csv'
    ORDER BY Year, tconst
''').df()

movie_year

,tconst,Year
0,tt0003740,1914
1,tt0008663,1917
2,tt0009369,1918
3,tt0010307,1919
4,tt0010600,1919
...,...,...
9995,tt9766294,2021
9996,tt9769668,2021
9997,tt9784798,2021
9998,tt9808510,2021


In [27]:
# Calculate the number of films worked on previously by the writers and directors
# Includes the current film

experience = con.execute('''
    SELECT tconst, Year, writer, director,
    COUNT(DISTINCT tconst) OVER(PARTITION BY writer ORDER BY Year, tconst) AS writer_experience,
    COUNT(DISTINCT tconst) OVER(PARTITION BY director ORDER BY Year, tconst) AS director_experience,
    FROM movie_year my
    LEFT JOIN writers ON writers.movie == my.tconst
    LEFT JOIN directors ON directors.movie == my.tconst
    ORDER BY Year, tconst
''').df()

experience

,tconst,Year,writer,director,writer_experience,director_experience
0,tt0003740,1914,nm0515385,nm0665163,1,1
1,tt0003740,1914,nm0758215,nm0665163,1,1
2,tt0003740,1914,nm0665163,nm0665163,1,1
3,tt0003740,1914,nm0195339,nm0665163,1,1
4,tt0008663,1917,nm0803705,nm0803705,1,1
...,...,...,...,...,...,...
27883,tt9784798,2021,nm3584219,nm3489851,1,1
27884,tt9808510,2021,nm8904180,nm8904180,1,1
27885,tt9808510,2021,nm10260663,nm8904180,1,1
27886,tt9808510,2021,nm9925241,nm8904180,1,1


In [28]:
# Merge movie data with the experience level of the writers and directors.
# For each film, calculate the number of writers/directors, average experience and total experience

data_experience = con.execute('''
    SELECT 
      ANY_VALUE(td.tconst) AS tconst, 
      ANY_VALUE(td.Year) AS Year, 
      ANY_VALUE(runtimeMinutes) AS runtimeMinutes,
      ANY_VALUE(ForeignFilm) AS ForeignFilm, 
      ANY_VALUE(n_words) AS n_words, 
      ANY_VALUE(numVotes) AS numVotes,
      COUNT(DISTINCT writer) AS n_writers,
      AVG(DISTINCT e.writer_experience) AS avgexp_writers,
      SUM(DISTINCT e.writer_experience) AS totexp_writers,
      COUNT(DISTINCT director) AS n_directors,
      AVG(DISTINCT e.director_experience) AS avgexp_directors,
      SUM(DISTINCT e.director_experience) AS totexp_directors,
      ANY_VALUE(label) AS label
    FROM all_training_data td
    LEFT JOIN experience e ON e.tconst == td.tconst
    GROUP BY td.tconst
''').df()

data_experience

,tconst,Year,runtimeMinutes,ForeignFilm,n_words,numVotes,n_writers,avgexp_writers,totexp_writers,n_directors,avgexp_directors,totexp_directors,label
0,tt0009369,1918,93,1,1,1119.0,2,1.000000,1.0,2,1.0,1.0,0
1,tt0011439,1920,79,1,4,2439.0,3,1.000000,1.0,1,1.0,1.0,1
2,tt0011841,1920,145,1,3,5376.0,5,1.000000,1.0,1,1.0,1.0,1
3,tt0012532,1921,150,0,4,NaN,3,1.500000,3.0,1,2.0,2.0,1
4,tt0127834,1921,62,0,4,1213.0,1,1.000000,1.0,1,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7954,tt5420210,2020,104,0,3,5737.0,3,2.666667,8.0,1,3.0,3.0,0
7955,tt6627168,2020,73,1,1,1411.0,3,1.000000,1.0,1,1.0,1.0,1
7956,tt11307814,2021,86,1,2,12409.0,2,1.500000,3.0,1,2.0,2.0,0
7957,tt4532038,2020,94,1,4,18446.0,3,1.500000,3.0,1,4.0,4.0,0


In [29]:
from sklearn.model_selection import train_test_split

# Split training data in train and test split

X = data_experience.drop(columns=['label'])
y = data_experience['label']

# Small test size because cross-validation (and want to train on more data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42, stratify=y)


In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Preprocessing pipeline
preprocessing = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
])

# Base model pipelines
models = {
    'LogisticRegression': Pipeline(steps=[('preprocessing', preprocessing), ('classifier', LogisticRegression(max_iter=1000))]),
    'DecisionTree': Pipeline(steps=[('preprocessing', preprocessing), ('classifier', DecisionTreeClassifier())]),
    'RandomForest': Pipeline(steps=[('preprocessing', preprocessing), ('classifier', RandomForestClassifier())]),
    'XGBoost': Pipeline(steps=[('preprocessing', preprocessing), ('classifier', xgb.XGBClassifier())])
    
}

hyperparameters = {
    'LogisticRegression': {
        'classifier__C': [0.01, 0.1, 1, 10]
    },
    'DecisionTree': {
        'classifier__max_depth': [None, 5, 10, 15, 20],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    },
    'RandomForest': {
        'classifier__n_estimators': [50, 100, 150, 200],
        'classifier__max_depth': [1, 3, 5, 7, 10],
    },
    'XGBoost': {
        'classifier__n_estimators': [50, 100, 150, 200],
        'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2],
        'classifier__max_depth': [2, 3, 4, 5],
        'classifier__subsample': [0.8, 1.0]
    }
}

best_models = {}
for model_name, model_pipeline in models.items():
    # Create a GridSearchCV object
    grid_search = GridSearchCV(model_pipeline, hyperparameters[model_name], cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
    
    # Fit the model
    grid_search.fit(X_train.drop(columns=['tconst']), y_train)
    
    # Store the best model
    best_models[model_name] = grid_search.best_estimator_
    
    # Print the best parameters and the best score
    print(f"{model_name} best parameters: {grid_search.best_params_}")
    print(f"{model_name} best score: {grid_search.best_score_:.4f}")
    
    # Use the best estimator to make predictions
    y_pred = grid_search.best_estimator_.predict(X_test.drop(columns=['tconst']))

    # Test if adjusting the threshold helps improve the accuracy (knowing dataset is balanced)
    y_proba = grid_search.best_estimator_.predict_proba(X_test.drop(columns=['tconst']))[:,1]
    threshold = np.median(y_proba)
    y_pred_th = (y_proba >= threshold).astype(int)

    # Calculate and print the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}% {accuracy_score(y_test, y_pred_th) * 100:.2f}%")

    # Print the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:")
    print(conf_matrix)

    # Print the classification report
    class_report = classification_report(y_test, y_pred)
    print("Classification Report:")
    print(class_report)

    # See how balanced the predictions are
    print (y_train.mean(), y_test.mean(), y_pred.mean(), threshold)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
LogisticRegression best parameters: {'classifier__C': 10}
LogisticRegression best score: 0.7254
Accuracy: 71.86% 72.74%
Confusion Matrix:
[[310  87]
 [137 262]]
Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.78      0.73       397
           1       0.75      0.66      0.70       399

    accuracy                           0.72       796
   macro avg       0.72      0.72      0.72       796
weighted avg       0.72      0.72      0.72       796

0.5013262599469496 0.5012562814070352 0.43844221105527637 0.443091041460555
Fitting 5 folds for each of 45 candidates, totalling 225 fits
DecisionTree best parameters: {'classifier__max_depth': 5, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2}
DecisionTree best score: 0.7361
Accuracy: 72.36% 71.11%
Confusion Matrix:
[[324  73]
 [147 252]]
Classification Report:
              precision    recall  f1-sco

In [31]:
# Load hidden validation and test sets and run through best performing prediction model
# TODO: Tidy this bit up

validation_data = con.execute('''
    SELECT tconst,
        REGEXP_REPLACE(TRANSLATE(LOWER(TRIM(primaryTitle)), 'áàãäåæßçéèêíîïñòóôöøớúûüý','aaaaaabceeeiiinoooooouuuy'),'[^a-zA-Z0-9 ]','','g') AS pTitle,
        REGEXP_REPLACE(TRANSLATE(LOWER(TRIM(originalTitle)),'áàãäåæßçéèêíîïñòóôöøớúûüý','aaaaaabceeeiiinoooooouuuy'),'[^a-zA-Z0-9 ]','','g') AS oTitle,
        CASE
           WHEN oTitle IS NULL THEN 0
           ELSE 1
        END AS ForeignFilm,
        LENGTH(pTitle) - LENGTH(REPLACE(pTitle, ' ', '')) + 1 AS n_words,
        CASE
            WHEN startYear LIKE '%N%' THEN endYear
            ELSE startYear
        END AS Year,
        runtimeMinutes,
        numVotes
        FROM 'big-data-course-2024-projects/imdb/validation_hidden.csv'
    ''').df().replace('\\N',np.nan)

test_data = con.execute('''
    SELECT tconst,
        REGEXP_REPLACE(TRANSLATE(LOWER(TRIM(primaryTitle)), 'áàãäåæßçéèêíîïñòóôöøớúûüý','aaaaaabceeeiiinoooooouuuy'),'[^a-zA-Z0-9 ]','','g') AS pTitle,
        REGEXP_REPLACE(TRANSLATE(LOWER(TRIM(originalTitle)),'áàãäåæßçéèêíîïñòóôöøớúûüý','aaaaaabceeeiiinoooooouuuy'),'[^a-zA-Z0-9 ]','','g') AS oTitle,
        CASE
           WHEN oTitle IS NULL THEN 0
           ELSE 1
        END AS ForeignFilm,
        LENGTH(pTitle) - LENGTH(REPLACE(pTitle, ' ', '')) + 1 AS n_words,
        CASE
            WHEN startYear LIKE '%N%' THEN endYear
            ELSE startYear
        END AS Year,
        runtimeMinutes,
        numVotes
        FROM 'big-data-course-2024-projects/imdb/test_hidden.csv'
    ''').df().replace('\\N',np.nan)

hidden_validation_data = con.execute('''
    SELECT
      ANY_VALUE(td.tconst) AS tconst,
      ANY_VALUE(td.Year) AS Year, 
      ANY_VALUE(runtimeMinutes) AS runtimeMinutes,
      ANY_VALUE(ForeignFilm) AS ForeignFilm, 
      ANY_VALUE(n_words) AS n_words, 
      ANY_VALUE(numVotes) AS numVotes,
      COUNT(DISTINCT writer) AS n_writers,
      AVG(DISTINCT e.writer_experience) AS avgexp_writers,
      SUM(DISTINCT e.writer_experience) AS totexp_writers,
      COUNT(DISTINCT director) AS n_directors,
      AVG(DISTINCT e.director_experience) AS avgexp_directors,
      SUM(DISTINCT e.director_experience) AS totexp_directors
    FROM validation_data td
    LEFT JOIN experience e ON e.tconst == td.tconst
    GROUP BY td.tconst
    ORDER BY td.tconst
''').df()

hidden_test_data = con.execute('''
    SELECT
      ANY_VALUE(td.tconst) AS tconst, 
      ANY_VALUE(td.Year) AS Year, 
      ANY_VALUE(runtimeMinutes) AS runtimeMinutes,
      ANY_VALUE(ForeignFilm) AS ForeignFilm, 
      ANY_VALUE(n_words) AS n_words, 
      ANY_VALUE(numVotes) AS numVotes,
      COUNT(DISTINCT writer) AS n_writers,
      AVG(DISTINCT e.writer_experience) AS avgexp_writers,
      SUM(DISTINCT e.writer_experience) AS totexp_writers,
      COUNT(DISTINCT director) AS n_directors,
      AVG(DISTINCT e.director_experience) AS avgexp_directors,
      SUM(DISTINCT e.director_experience) AS totexp_directors
    FROM test_data td
    LEFT JOIN experience e ON e.tconst == td.tconst
    GROUP BY td.tconst
    ORDER BY td.tconst
''').df()



In [32]:
# Choose best performing model

valid_pred = best_models['RandomForest'].predict(hidden_validation_data.drop(columns=['tconst']))
test_pred = best_models['RandomForest'].predict(hidden_test_data.drop(columns=['tconst']))

print (valid_pred.mean(), test_pred.mean())

# Add predicted column
hidden_validation_data['pred_label'] = valid_pred
hidden_test_data['pred_label'] = test_pred

# Save to test file for prediction upload
np.savetxt('valid_predictions.txt', np.where(valid_pred == 1, 'True', 'False'), fmt='%s', newline='\n')
np.savetxt('test_predictions.txt', np.where(test_pred == 1, 'True', 'False'), fmt='%s', newline='\n')

0.41465968586387436 0.44659300184162065


In [33]:
# Validate that the labels match up with the correct movie title (in case order has been messed around)

df_valid = pd.read_csv('big-data-course-2024-projects/imdb/validation_hidden.csv')
df_test = pd.read_csv('big-data-course-2024-projects/imdb/test_hidden.csv')

print ((df_valid['tconst'] == hidden_validation_data['tconst']).all(),
       (df_test['tconst'] == hidden_test_data['tconst']).all())

True True
